In [31]:
import pandas as pd
df50_index= pd.read_excel(r"上证50指数（CCt）.xlsx", engine='openpyxl')
df50_index.head()


,指标名称,上证50指数
0,频率,日
1,单位,点
2,指标ID,M002845716
3,2022-12-30,2635.25
4,2022-12-29,2622.07


In [32]:
#drop first 3 row
df50_index = df50_index.drop([0,1,2])
#change column name
df50_index.columns=['date', 'index_level']

In [33]:
df50_index['date']=pd.to_datetime(df50_index['date'])
df50_index.set_index("date", inplace=True)
weekly_groups=df50_index.groupby(pd.Grouper(freq='W'))
#find the last day of each week
df_index_weekly=weekly_groups.last()
df_index_weekly
df_index_weekly.isnull().sum()
df_index_weekly.dropna(inplace=True)

In [34]:

#remove first row
df_index_weekly = df_index_weekly.iloc[1:]
df_index_weekly

,index_level
date,
2013-04-21,1827.98
2013-04-28,1766.04
2013-05-05,1795.49
2013-05-12,1819.12
2013-05-19,1843.81
...,...
2022-12-04,2614.61
2022-12-11,2706.74
2022-12-18,2688.29


In [35]:
df_index_weekly
#to csv file
df_index_weekly.to_csv('df_index_weekly.csv')

In [36]:
#add index week1 week2 week3...
import numpy as np
df_index_weekly['week'] = range(1, len(df_index_weekly) + 1)
df_index_weekly.set_index("week", inplace=True)
df_index_weekly


,index_level
week,
1,1827.98
2,1766.04
3,1795.49
4,1819.12
5,1843.81
...,...
495,2614.61
496,2706.74
497,2688.29


In [37]:
#last 52 weeks' max value for weekly_indexLevel Dataset
df_index_weekly['max_52'] = None
for i in range(52, len(df_index_weekly)+1):
    previous_52 = df_index_weekly.loc[i-52:i-1, 'index_level']  
    max_value = previous_52.max()  
    df_index_weekly.at[i, 'max_52'] = max_value  

df_index_weekly

,index_level,max_52
week,,
1,1827.98,None
2,1766.04,None
3,1795.49,None
4,1819.12,None
5,1843.81,None
...,...,...
495,2614.61,3353.63
496,2706.74,3353.63
497,2688.29,3353.63


In [38]:
#calculate ML = index_level/(max of last 52 weeks) and create a new column
df_index_weekly['ML'] = df_index_weekly['index_level']/df_index_weekly['max_52']
df_index_weekly

,index_level,max_52,ML
week,,,
1,1827.98,None,NaN
2,1766.04,None,NaN
3,1795.49,None,NaN
4,1819.12,None,NaN
5,1843.81,None,NaN
...,...,...,...
495,2614.61,3353.63,0.779636
496,2706.74,3353.63,0.807108
497,2688.29,3353.63,0.801606


In [39]:
#find the mean and standard deviation of ML for past 52 weeks
df_index_weekly['mean_52'] = None
df_index_weekly['std_52'] = None
for i in range(104, len(df_index_weekly)+1):
    previous_52 = df_index_weekly.loc[i-52:i-1, 'ML']  
    mean_value = previous_52.mean()  
    std_value = previous_52.std()  
    df_index_weekly.at[i, 'mean_52'] = mean_value  
    df_index_weekly.at[i, 'std_52'] = std_value

df_index_weekly

,index_level,max_52,ML,mean_52,std_52
week,,,,,
1,1827.98,None,NaN,None,None
2,1766.04,None,NaN,None,None
3,1795.49,None,NaN,None,None
4,1819.12,None,NaN,None,None
5,1843.81,None,NaN,None,None
...,...,...,...,...,...
495,2614.61,3353.63,0.779636,0.793624,0.035088
496,2706.74,3353.63,0.807108,0.79328,0.035137
497,2688.29,3353.63,0.801606,0.793586,0.035188


In [40]:
df_index_weekly['mean-1.5*std'] = df_index_weekly['mean_52'] - 1.5*df_index_weekly['std_52']
df_index_weekly['CC'] = np.where(df_index_weekly['ML'] < df_index_weekly['mean-1.5*std'], 1, 0)
df_index_weekly

,index_level,max_52,ML,mean_52,std_52,mean-1.5*std,CC
week,,,,,,,
1,1827.98,None,NaN,None,None,NaN,0
2,1766.04,None,NaN,None,None,NaN,0
3,1795.49,None,NaN,None,None,NaN,0
4,1819.12,None,NaN,None,None,NaN,0
5,1843.81,None,NaN,None,None,NaN,0
...,...,...,...,...,...,...,...
495,2614.61,3353.63,0.779636,0.793624,0.035088,0.740992,0
496,2706.74,3353.63,0.807108,0.79328,0.035137,0.740575,0
497,2688.29,3353.63,0.801606,0.793586,0.035188,0.740804,0


In [41]:
#remove first 104 rows
df_index_weekly = df_index_weekly.iloc[104:]
df_index_weekly

,index_level,max_52,ML,mean_52,std_52,mean-1.5*std,CC
week,,,,,,,
105,3235.33,2986.61,1.083278,0.934589,0.085595,0.806198,0
106,3223.96,3235.33,0.996486,0.939555,0.0866,0.809654,0
107,3250.49,3235.33,1.004686,0.943062,0.0851,0.815412,0
108,3110.46,3250.49,0.95692,0.946792,0.08342,0.821663,0
109,3060.41,3250.49,0.941523,0.949751,0.080914,0.82838,0
...,...,...,...,...,...,...,...
495,2614.61,3353.63,0.779636,0.793624,0.035088,0.740992,0
496,2706.74,3353.63,0.807108,0.79328,0.035137,0.740575,0
497,2688.29,3353.63,0.801606,0.793586,0.035188,0.740804,0


In [42]:
#to csv
df_index_weekly.to_csv(r'50_IL_Weekly_ML.csv', index = True, header=True)

In [43]:
df_index_weekly.describe()

,index_level,CC
count,395.000000,395.000000
mean,2779.862456,0.197468
std,427.842100,0.398594
min,1955.400000,0.000000
25%,2429.380000,0.000000
50%,2775.810000,0.000000
75%,3061.395000,0.000000
max,4028.530000,1.000000
